Importing Libraries

In [91]:
import pandas as pd
import numpy as np
from datetime import datetime

Importing Dataset

In [92]:
dataframes = []
for i in range(1,13):
    if i==6:
        continue
    
    if i != 4:
        df = pd.read_csv(f'{i}_2024_113.csv', index_col=0)
    else:
        df = pd.read_csv('2_2024_to_4_2024_113.csv', index_col=0)

    df.index.name = 'bulk_id'
    df['time_stamp'] = df['time_stamp'].str.replace('T',' ')
    df['time_stamp'] = pd.to_datetime(df['time_stamp'])
    dataframes.append(df)

df = pd.concat(dataframes, verify_integrity=True)
df = df.sort_values(by='time_stamp', ascending=True)
df.head()

,car_reg_no,vehicle_speed,heading,distance,altitude,o_s1_b1_voltage,o_s2_b2_voltage,o_s1_current,calculated_engine_load,engine_rpm,...,egr_error,absolute_barometric_pressure,engine_coolant_temperature,intake_air_temperature,catalyst_temperature_b1_s1,catalyst_temperature_b1_s2,ambient_air_temperature,time_stamp,latitude,longitude
bulk_id,,,,,,,,,,,,,,,,,,,,,
285444,113,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-02-05 11:05:07,0.0,0.0
285447,113,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-02-05 11:05:09,0.0,0.0
285450,113,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-02-05 11:05:11,0.0,0.0
285453,113,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-02-05 11:05:15,0.0,0.0
285456,113,0.0,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2021-02-05 11:05:17,0.0,0.0


Dropping the null values

In [93]:
df = df.dropna(axis=1, how='all') #dropping the columns which contains all null values

# #repeated columns which one to keep?
print(df['o_s1_b1_fuel_air_equivalence_ratio'].describe())
print(df['o_s1_b1_fuelair_equivalence_ratio'].describe()) #contains all zeros so removing it

count    702207.000000
mean         14.276346
std           0.808520
min           7.299800
25%          14.123500
50%          14.700000
75%          14.700000
max          15.234700
Name: o_s1_b1_fuel_air_equivalence_ratio, dtype: float64
count    701891.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: o_s1_b1_fuelair_equivalence_ratio, dtype: float64


Dropping repeated column

In [94]:
df = df.drop('o_s1_b1_fuelair_equivalence_ratio',axis=1)

Removing null values

In [95]:
print(df.isna().sum())

df['engine_rpm'] = df['engine_rpm'].fillna(0) #filling the null values with zero as we observed that when engine_rpm was null vehicle_speed was 0/null and other values were also null, so the engine would be turned off

df[df['engine_rpm']==0] = df[df['engine_rpm']==0].fillna(0) #when engine is turned off all other components are also turned off

print('Data after filling the engine off readings')
df.isna().sum()

car_reg_no                                 0
vehicle_speed                              0
heading                                    0
altitude                                   0
o_s1_b1_voltage                       759651
o_s2_b2_voltage                        52425
o_s1_current                           93601
calculated_engine_load                 53236
engine_rpm                             53865
spark_advance                          54527
absolute_load_value                    98848
throttle_position                      55105
relative_throttle_position             55330
absolute_throttle_position             99486
ap_pos_d                               99992
ap_pos_e                              100314
commanded_evaporative_purge            56646
o_s1_b1_fuel_air_equivalence_ratio    100854
short_term_fuel_trim_b1                 6598
long_term_fuel_trim_b1                  6598
intake_manifold_absolute_pressure     759719
mass_air_flow_rate                     58522
absolute_b

car_reg_no                                 0
vehicle_speed                              0
heading                                    0
altitude                                   0
o_s1_b1_voltage                       698366
o_s2_b2_voltage                           27
o_s1_current                             642
calculated_engine_load                     0
engine_rpm                                 0
spark_advance                            662
absolute_load_value                     1700
throttle_position                       1240
relative_throttle_position              1458
absolute_throttle_position              2324
ap_pos_d                                2826
ap_pos_e                                3145
commanded_evaporative_purge             2756
o_s1_b1_fuel_air_equivalence_ratio      3674
short_term_fuel_trim_b1                    0
long_term_fuel_trim_b1                     0
intake_manifold_absolute_pressure     698397
mass_air_flow_rate                      4604
absolute_b

Removing remaining null values

In [96]:
df.fillna(df[df!=0].mean(),inplace=True) #replacing null values with mean value

print(df['o_s1_current'].unique()) #all zeros
print(df['catalyst_temperature_b1_s1'].unique()) #all zeros
print(df['catalyst_temperature_b1_s2'].unique()) #all zeros

# #dropping the columns
df = df.drop(['o_s1_current','catalyst_temperature_b1_s1','catalyst_temperature_b1_s2'], axis=1)

print('\nCleaned Data:')
print(df.isna().sum())
print('No. of rows:',len(df))

[ 0. nan]
[ 0. nan]
[ 0. nan]

Cleaned Data:
car_reg_no                            0
vehicle_speed                         0
heading                               0
altitude                              0
o_s1_b1_voltage                       0
o_s2_b2_voltage                       0
calculated_engine_load                0
engine_rpm                            0
spark_advance                         0
absolute_load_value                   0
throttle_position                     0
relative_throttle_position            0
absolute_throttle_position            0
ap_pos_d                              0
ap_pos_e                              0
commanded_evaporative_purge           0
o_s1_b1_fuel_air_equivalence_ratio    0
short_term_fuel_trim_b1               0
long_term_fuel_trim_b1                0
intake_manifold_absolute_pressure     0
mass_air_flow_rate                    0
absolute_barometric_pressure          0
engine_coolant_temperature            0
intake_air_temperature             

Filtering for 2024 data

In [97]:
df = df.sort_values(by='time_stamp')
start = pd.to_datetime('2024-01-01 00:00:00')
end = pd.to_datetime('2024-12-31 23:59:59')

df = df[(df['time_stamp'] >= start) & (df['time_stamp']<=end)]

print(df['time_stamp'].iloc[0])
print(df['time_stamp'].iloc[-1])
print('No. of rows:',len(df))

2024-01-01 08:44:34
2024-12-28 23:48:39
No. of rows: 639483


Checking for duplicate values

In [98]:
duplicate = {}
for column in df.columns:
    duplicate[column] = df[column].duplicated()

duplicate = pd.DataFrame(duplicate)
print('No. of duplicate values in each column')
duplicate.sum()

No. of duplicate values in each column


car_reg_no                            639482
vehicle_speed                         639362
heading                               598475
altitude                              637459
o_s1_b1_voltage                       639332
o_s2_b2_voltage                       639362
calculated_engine_load                639269
engine_rpm                            636845
spark_advance                         639331
absolute_load_value                   639307
throttle_position                     639316
relative_throttle_position            639328
absolute_throttle_position            639339
ap_pos_d                              639374
ap_pos_e                              639372
commanded_evaporative_purge           639236
o_s1_b1_fuel_air_equivalence_ratio    638915
short_term_fuel_trim_b1               639427
long_term_fuel_trim_b1                639444
intake_manifold_absolute_pressure     639414
mass_air_flow_rate                    636978
absolute_barometric_pressure          639477
engine_coo

Analyzing duplicated time stamp data

In [100]:
duplicates = df['time_stamp'][df['time_stamp'].duplicated(keep=False)]

duplicates_df = df[df['time_stamp'].isin(duplicates)]
print(len(duplicates_df))

identical = duplicates_df.groupby('time_stamp').apply(lambda group: group.nunique().max() == 1)

identical_groups = identical.sum()
total_group = identical.count()

print("Duplicated timestamps with identical rows across columns:")
print(identical)

print("\nNumber of identical groups:",identical_groups)
print("Total number of groups:", total_group)


274977
Duplicated timestamps with identical rows across columns:
time_stamp
2024-01-01 22:08:56    True
2024-01-01 22:08:58    True
2024-01-01 22:09:00    True
2024-01-01 22:09:06    True
2024-01-01 22:09:08    True
                       ... 
2024-12-13 16:19:26    True
2024-12-13 16:19:28    True
2024-12-13 16:19:30    True
2024-12-13 16:19:32    True
2024-12-13 16:19:34    True
Length: 50543, dtype: bool

Number of identical groups: 50543
Total number of groups: 50543


C:\Users\Moiz\AppData\Local\Temp\ipykernel_3572\1766386042.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  identical = duplicates_df.groupby('time_stamp').apply(lambda group: group.nunique().max() == 1)


Removing duplicated entries

In [101]:
df = df.drop_duplicates(subset=['time_stamp'])
print('No. of rows after deleting duplicate entries:',len(df))

No. of rows after deleting duplicate entries: 415049


Exporting the cleaned data frame

In [102]:
df.to_csv('cleaned_data_2024.csv')